In [22]:
import numpy as np
import pandas as pd


In [33]:
# Define the KNN class
class KNN:
    def __init__(self, k=5, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # Ensure that y is an integer array (class labels)
        self.X_train = np.array(X)
        self.y_train = np.array(y, dtype=int)  # Convert y to integer type

    def predict(self, X):
        # Convert the test data to a NumPy array
        X = np.array(X)
        predictions = []
        
        # For each test point, compute distances to all training points
        for x in X:
            distances = [self.compute_distance(x, x_train) for x_train in self.X_train]
            k_indices = np.argsort(distances)[:self.k]  # Get indices of k nearest neighbors
            k_nearest_labels = self.y_train[k_indices]
            most_common = np.bincount(k_nearest_labels).argmax()  # Get most frequent label
            predictions.append(most_common)
        
        return np.array(predictions)

    def compute_distance(self, X1, X2):
        # Compute Euclidean or Manhattan distance based on the chosen metric
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1 - X2) ** 2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1 - X2))



In [24]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    # Load the train and test datasets
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Separate features (X) and target (y) in training data
    X_train = train_data.drop(columns=['Exited', 'id', 'CustomerId', 'Surname'])  # Assuming 'Exited' is the target column
    y_train = train_data['Exited']
    
    # Use the test data as it is (no target in the test set)
    X_test = test_data.drop(columns=['id', 'CustomerId', 'Surname'])
    
    # Handle missing values (only for numerical columns)
    num_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
    X_train[num_cols] = X_train[num_cols].fillna(X_train[num_cols].mean())
    X_test[num_cols] = X_test[num_cols].fillna(X_test[num_cols].mean())
    
    # Manually encode categorical variables
    for col in X_train.select_dtypes(include=['object']).columns:
        unique_values = X_train[col].unique()
        value_map = {value: idx for idx, value in enumerate(unique_values)}
        X_train[col] = X_train[col].map(value_map)
        X_test[col] = X_test[col].map(value_map)  # Ensure the same mapping is used for test data
    
    # Manual feature scaling (standardization)
    X_train_scaled = (X_train - X_train.mean()) / X_train.std()
    X_test_scaled = (X_test - X_train.mean()) / X_train.std()  # Use train set statistics to scale test set
    
    # Return the processed data
    return X_train_scaled.values, y_train.values, X_test_scaled.values




In [25]:
def kfold_split(X, y, n_splits=5):
    fold_size = len(X) // n_splits
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    for i in range(n_splits):
        val_indices = indices[i * fold_size:(i + 1) * fold_size]
        train_indices = np.concatenate((indices[:i * fold_size], indices[(i + 1) * fold_size:]))
        
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]
        
        yield X_train, X_val, y_train, y_val

def roc_auc_score(y_true, y_pred):
    # Combine the true labels and predicted values
    sorted_indices = np.argsort(y_pred)
    y_true_sorted = y_true[sorted_indices]
    y_pred_sorted = y_pred[sorted_indices]
    
    # Calculate the true positive rate (TPR) and false positive rate (FPR)
    tpr = []
    fpr = []
    pos_count = np.sum(y_true)
    neg_count = len(y_true) - pos_count
    tp = 0
    fp = 0
    
    for i in range(len(y_true)):
        if y_true_sorted[i] == 1:
            tp += 1
        else:
            fp += 1
        tpr.append(tp / pos_count)
        fpr.append(fp / neg_count)
    
    # Calculate the area under the curve using the trapezoidal rule
    auc = 0.0
    for i in range(1, len(tpr)):
        auc += (fpr[i] - fpr[i - 1]) * (tpr[i] + tpr[i - 1]) / 2
    
    return auc


In [26]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    auc_scores = []
    
    # Perform k-fold cross-validation using the kfold_split function
    for X_train, X_val, y_train, y_val in kfold_split(X, y, n_splits):
        # Train the KNN model on the training set
        knn.fit(X_train, y_train)
        
        # Get predictions on the validation set
        y_pred = knn.predict(X_val)
        
        # Calculate the ROC AUC score using the custom roc_auc_score function
        auc = roc_auc_score(y_val, y_pred)
        auc_scores.append(auc)
    
    # Return the average ROC AUC score across all folds
    return np.mean(auc_scores)


In [27]:
X, y, X_test = preprocess_data('/Users/kenneytran/Downloads/CS506-Assignment5/train.csv', '/Users/kenneytran/Downloads/CS506-Assignment5/test.csv')


In [32]:
# Use only a portion of the data for hyperparameter tuning to save time
X_subset = X[:1000]
y_subset = y[:1000]

# Perform hyperparameter tuning on a subset
best_k = None
best_metric = None
best_score = -1

for k in [3, 5, 7, 9]:  # Try different values of k
    for metric in ['euclidean', 'manhattan']:  # Try different distance metrics
        knn = KNN(k=k, distance_metric=metric)
        score = cross_validate(X_subset, y_subset, knn)  # Perform cross-validation on the subset
        
        print(f"k={k}, metric={metric}, AUC={score}")
        
        if score > best_score:
            best_k = k
            best_metric = metric
            best_score = score

print(f"Best parameters: k={best_k}, metric={best_metric}, AUC={best_score}")


# Train on full dataset with optimal hyperparameters
knn = KNN(k=best_k, distance_metric=best_metric)
knn.fit(X, y)

# Make predictions on the test set
test_predictions = knn.predict(X_test)

# Save test predictions
pd.DataFrame({'id': pd.read_csv('/Users/kenneytran/Downloads/CS506-Assignment5/test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)


k=3, metric=euclidean, AUC=0.2425122703801718
k=3, metric=manhattan, AUC=0.29688523365883546
k=5, metric=euclidean, AUC=0.31903758397505566
k=5, metric=manhattan, AUC=0.26746175443054254
k=7, metric=euclidean, AUC=0.3120559826166973
k=7, metric=manhattan, AUC=0.24037564351142587
k=9, metric=euclidean, AUC=0.30677924202890494
k=9, metric=manhattan, AUC=0.26669756643652637
k=15, metric=euclidean, AUC=0.2742530515910758
k=15, metric=manhattan, AUC=0.2753447184727934
k=20, metric=euclidean, AUC=0.2946009711563314
k=20, metric=manhattan, AUC=0.3387296188864217
Best parameters: k=20, metric=manhattan, AUC=0.3387296188864217
Validation Accuracy: 0.8860 with n_neighbors=20
